<a href="https://colab.research.google.com/github/r-vikrantsagar/Machine-Learning/blob/main/Med_Prescription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Load the datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Display the first few rows of each dataframe to understand the structure
train.head(), test.head(), sample_submission.head()


NameError: name 'sample_submission' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

# Convert launch_date to datetime
train['launch_date'] = pd.to_datetime(train['launch_date'], format='%d-%b-%y')
test['launch_date'] = pd.to_datetime(test['launch_date'], format='%d-%b-%y')

# Extract year, month, and day from launch_date
train['launch_year'] = train['launch_date'].dt.year
train['launch_month'] = train['launch_date'].dt.month
train['launch_day'] = train['launch_date'].dt.day

test['launch_year'] = test['launch_date'].dt.year
test['launch_month'] = test['launch_date'].dt.month
test['launch_day'] = test['launch_date'].dt.day

# Drop the original launch_date column
train = train.drop(columns=['launch_date'])
test = test.drop(columns=['launch_date'])

# Define feature columns
numeric_features = ['market_value', 'launch_year', 'launch_month', 'launch_day']
categorical_features = ['disease_type']
text_features = 'medicine_review'

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('text', TfidfVectorizer(), text_features)
    ])

# Define the model
model = make_pipeline(preprocessor, Ridge(alpha=1.0))

# Split the training data into training and validation sets
X = train.drop(columns=['score'])
y = train['score']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Validate the model
y_val_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
score = max(0, 100 - rmse)

# Print the validation score
score


ValueError: time data "April 29, 2017" doesn't match format "%d-%b-%y", at position 8. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
from dateutil import parser

# Function to parse dates with multiple formats
def parse_date(date_str):
    for fmt in ("%d-%b-%y", "%B %d, %Y"):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.to_datetime(date_str)

# Apply the function to the launch_date column
train['launch_date'] = train['launch_date'].apply(parse_date)
test['launch_date'] = test['launch_date'].apply(parse_date)

# Extract year, month, and day from launch_date
train['launch_year'] = train['launch_date'].dt.year
train['launch_month'] = train['launch_date'].dt.month
train['launch_day'] = train['launch_date'].dt.day

test['launch_year'] = test['launch_date'].dt.year
test['launch_month'] = test['launch_date'].dt.month
test['launch_day'] = test['launch_date'].dt.day

# Drop the original launch_date column
train = train.drop(columns=['launch_date'])
test = test.drop(columns=['launch_date'])

# Define feature columns
numeric_features = ['market_value', 'launch_year', 'launch_month', 'launch_day']
categorical_features = ['disease_type']
text_features = 'medicine_review'

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('text', TfidfVectorizer(), text_features)
    ])

# Define the model
model = make_pipeline(preprocessor, Ridge(alpha=1.0))

# Split the training data into training and validation sets
X = train.drop(columns=['score'])
y = train['score']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Validate the model
y_val_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
score = max(0, 100 - rmse)

# Print the validation score
score


ValueError: Found unknown categories [9, 12, 35, 36, 39, 69, 149, 161, 198, 202, 257, 270, 274, 292, 320, 325, 341, 342, 344, 363, 373, 377, 382, 416, 426, 432, 450, 467, 520, 527, 536, 540, 553, 560, 613, 625, 675, 696] in column 0 during transform

In [ ]:
# Update the column transformer to handle unknown categories
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('text', TfidfVectorizer(), text_features)
    ])

# Define the model
model = make_pipeline(preprocessor, Ridge(alpha=1.0))

# Train the model
model.fit(X_train, y_train)

# Validate the model
y_val_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
score = max(0, 100 - rmse)

# Print the validation score
score


70.48687585441658

In [ ]:
# Validate the model
y_val_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
score = max(0, 100 - rmse)

# Predict the usefulness scores for the test set
test_predictions = model.predict(test)

# Prepare the submission file
submission = pd.DataFrame({
    'medicine_no': test['medicine_no'],
    'score': test_predictions
})

# Ensure the submission file matches the required format
submission.to_csv('submission.csv', index=False)

# Display the first few rows of the submission file to the user
submission.head()


,medicine_no,score
0,MD-49997,53.294884
1,MD-49998,36.755559
2,MD-49999,36.426887
3,MD-50000,31.925697
4,MD-50001,13.476798


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
from dateutil import parser


# Load the datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Convert the 'launch_date' column to datetime format without specifying the format
train['launch_date'] = pd.to_datetime(train['launch_date'], errors='coerce')

# Define feature columns
numeric_features = ['market_value']
categorical_features = ['disease_type']
text_features = 'medicine_review'

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('text', TfidfVectorizer(stop_words= 'english'), text_features)
    ])

# Define the model
model = make_pipeline(preprocessor, Ridge(alpha=1.0))


# Split the training data into training and validation sets
X = train.drop(columns=['score'])
y = train['score']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Validate the model
y_val_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
score = max(0, 100 - rmse)


# Predict the usefulness scores for the test set
test_predictions = model.predict(test)

# Prepare the submission file
submission = pd.DataFrame({
    'medicine_no': test['medicine_no'],
    'score': np.round(test_predictions).astype(int)
})

# Ensure the submission file matches the required format
submission.to_csv('submission.csv', index=False)

# Display the first few rows of the submission file to the user
print(submission.head())

<ipython-input-30-d184e18be0ae>:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['launch_date'] = pd.to_datetime(train['launch_date'], errors='coerce')


  medicine_no  score
0    MD-49997     38
1    MD-49998     47
2    MD-49999     38
3    MD-50000     31
4    MD-50001     11


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
from dateutil import parser

# Function to parse dates with multiple formats
def parse_date(date_str):
    for fmt in ("%d-%b-%y", "%B %d, %Y"):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.to_datetime(date_str)

# Load the datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Convert launch_date to datetime using the custom function
train['launch_date'] = train['launch_date'].apply(parse_date)
test['launch_date'] = test['launch_date'].apply(parse_date)

# Extract year, month, and day from launch_date
train['launch_year'] = train['launch_date'].dt.year
train['launch_month'] = train['launch_date'].dt.month
train['launch_day'] = train['launch_date'].dt.day

test['launch_year'] = test['launch_date'].dt.year
test['launch_month'] = test['launch_date'].dt.month
test['launch_day'] = test['launch_date'].dt.day

# Drop the original launch_date column
train = train.drop(columns=['launch_date'])
test = test.drop(columns=['launch_date'])

# Define feature columns
numeric_features = ['market_value', 'launch_year', 'launch_month', 'launch_day']
categorical_features = ['disease_type']
text_features = 'medicine_review'

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('text', TfidfVectorizer(), text_features)
    ])

# Define the model with reduced complexity
model_rf = make_pipeline(preprocessor, RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42))

# Split the training data into training and validation sets
X = train.drop(columns=['score'])
y = train['score']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model_rf.fit(X_train, y_train)

# Validate the model
y_val_pred = model_rf.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
score = max(0, 100 - rmse)

# Print the validation RMSE and score
print("Validation RMSE:", rmse)
print("Validation Score:", score)

# Predict the usefulness scores for the test set
test_predictions = model_rf.predict(test)

# Prepare the submission file
submission = pd.DataFrame({
    'medicine_no': test['medicine_no'],
    'score': test_predictions
})

# Ensure the submission file matches the required format
submission.to_csv('submission.csv', index=False)

# Display the first few rows of the submission file to the user
print(submission.head())


Validation RMSE: 28.921655716305327
Validation Score: 71.07834428369468
  medicine_no      score
0    MD-49997  50.935425
1    MD-49998  26.473147
2    MD-49999  21.754482
3    MD-50000  16.532694
4    MD-50001   4.336079


### Final **Edit**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from dateutil import parser

# Load the datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Convert the 'launch_date' column to datetime format without specifying the format
train['launch_date'] = pd.to_datetime(train['launch_date'], errors='coerce')

# Display the first few rows to verify the conversion
train.head()



<ipython-input-2-d4ffe0942aef>:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train['launch_date'] = pd.to_datetime(train['launch_date'], errors='coerce')


,medicine_no,disease_type,medicine_review,market_value,launch_date,score
0,MD-22117,677,I read so many horrible reviews of this that I...,10,2016-03-19,19
1,MD-22118,121,"When I took the pill, I had a hard time rememb...",8,2013-10-28,21
2,MD-22119,200,This is the only antidepressant that was bene...,9,2017-06-01,34
3,MD-22120,455,I took Alli for 2 months when it first came ou...,7,2008-05-13,98
4,MD-22121,404,About ten days or two weeks after starting Est...,10,2009-12-13,43


In [ ]:
# Define feature columns
numeric_features = ['market_value','launch_date']
categorical_features = ['disease_type']
text_features = 'medicine_review'

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('text', TfidfVectorizer(), text_features)
    ])


In [ ]:
# Update the column transformer to handle unknown categories
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('text', TfidfVectorizer(), text_features)
    ])

# Define the model
model = make_pipeline(preprocessor, Ridge(alpha=1.0))

# Split the training data into training and validation sets
X = train.drop(columns=['score'])
y = train['score']
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

DTypePromotionError: The DType <class 'numpy.dtypes.Int64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.DateTime64DType'>)

In [ ]:
# Train the model
model.fit(X_train, y_train)

# Validate the model
y_val_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
score = max(0, 100 - rmse)

# Print the validation RMSE and score
print("Validation RMSE:", rmse)
print("Validation Score:", score)

# Predict the usefulness scores for the test set
test_predictions = model.predict(test)

# Prepare the submission file
submission = pd.DataFrame({
    'medicine_no': test['medicine_no'],
    'score': test_predictions
})

# Ensure the submission file matches the required format
submission.to_csv('submission.csv', index=False)

# Display the first few rows of the submission file to the user
print(submission.head())


DTypePromotionError: The DType <class 'numpy.dtypes.Int64DType'> could not be promoted by <class 'numpy.dtypes.DateTime64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.Int64DType'>, <class 'numpy.dtypes.DateTime64DType'>)